# L3: Advanced queries with Agents

In this lesson, you are going to do more advanced querying.

### Import required dependencies and load the API key

In [1]:
import * as mod from "https://deno.land/std@0.213.0/dotenv/mod.ts";
import { 
    Document, 
    VectorStoreIndex, 
    SimpleDirectoryReader,
    RouterQueryEngine,
    OpenAIAgent,
    QueryEngineTool,
    FunctionTool,
} from "npm:llamaindex@0.1.18"
const keys = await mod.load({export:true}) // read API key from .env

Let's create 2 different data sources that answer different types of questions:
- The first is the same example as in lesson 1, an essay by Dan Abramov about his career.
- The second is the wikipedia page content about React.

**Note**: To access the data used in this lesson, go to `File` and click on `Open` 

In [2]:
const documents1 = await new SimpleDirectoryReader().loadData({directoryPath: "./data"})
const index1 = await VectorStoreIndex.fromDocuments(documents1)
const queryEngine1 = index1.asQueryEngine()

In [3]:
let response1 = await queryEngine1.query({query: "What did the author do in college?"})
console.log(response1.toString())

In college, the author initially struggled with the new environment and differences from their previous educational experience. However, after adapting to the learning methods, they excelled academically. They also started learning full-stack development on their own, completed some projects, and then began their Linux journey, starting with basic commands and small projects. Eventually, they became a Linux system administrator, solving problems for about a year before transitioning into DevOps engineering. They learned about Docker, Kubernetes, Chef, Puppet, and Ansible, eventually becoming a junior DevOps engineer. They then transitioned into cloud computing, primarily working with Google Cloud and obtaining certifications in Oracle Cloud and Google Cloud. Additionally, they started exploring AI and completed small projects in that field. Currently, they work at Accenture as a developer and continue to work on AI and cloud-based projects for fun on weekends and off days.


In [4]:
const documents2 = await new SimpleDirectoryReader().loadData({directoryPath: "./data2"})
const index2 = await VectorStoreIndex.fromDocuments(documents2)
const queryEngine2 = index2.asQueryEngine()

In [5]:
let response2 = await queryEngine2.query({query:"What is React?"})
console.log(response2.toString())

React is a JavaScript library for building user interfaces. It was officially open-sourced in May 2013 at JSConf US, marking a significant turning point in its adoption and growth. React enables developers to create dynamic and interactive user interfaces for web applications. Additionally, React Native, which was announced in February 2015 and open-sourced in March 2015, allows for native Android, iOS, and Universal Windows Platform development using React. The library underwent a significant update with the introduction of React Fiber in April 2017, which improved the rendering algorithms and performance of React applications. Subsequent versions, such as React 16.0 in September 2017 and React 18 in March 2022, have introduced new features and enhancements to the library.


### Create a router query engine

In [6]:
const queryEngine = await RouterQueryEngine.fromDefaults({
  queryEngineTools: [
    {
      queryEngine: queryEngine1,
      description: "Useful for questions about Dan Abramov",
    },
    {
      queryEngine: queryEngine2,
      description: "Useful for questions about the React library",
    },
  ],
});

### Ask it a question

In [7]:
let response3 = await queryEngine.query({query: "What is React?"})
console.log(response3.toString())

React is a JavaScript library for building user interfaces. It was officially open-sourced in May 2013 at JSConf US, marking a significant turning point in its adoption and growth. React enables developers to create interactive and dynamic user interfaces for web applications. Additionally, React Native, an extension of React, allows for native Android, iOS, and Universal Windows Platform development with React. The library underwent significant improvements with the introduction of React Fiber, a new set of internal algorithms for rendering, announced by Facebook in April 2017. This new rendering system aimed to enhance performance and enable smoother on-screen rendering by breaking down animations and page structures into segments that can be updated separately. Subsequent versions of React, such as React 16.0 released in September 2017 and React 18 released in March 2022, introduced new features and improvements to the library.


In [8]:
let response4 = await queryEngine.query({query: "What did Dan Abramov do in college?"})
console.log(response4.toString())

Dan Abramov started college in 2019 in Chennai, India. Initially finding it challenging due to the new environment and differences from his previous educational experience, he adapted to the learning methods and excelled academically after the first year. During his college years, he started learning full-stack development on his own to compensate for the lacking curriculum. He also began his Linux journey, starting with basic commands and small projects, eventually becoming a Linux system administrator and then transitioning into DevOps engineering. Over the next few years, he gained experience in cloud computing, primarily working with Google Cloud, and also explored AI, completing small projects in that field. Currently, Dan Abramov works at Accenture as a developer and continues to work on AI and cloud-based projects for fun on weekends and off days.


### Define a simple function

In [9]:
function sumNumbers({a,b}) {
  return a + b;
}

### Explain the function in a JSON structure

In [10]:
const sumJSON = {
  type: "object",
  properties: {
    a: {
      type: "number",
      description: "The first number",
    },
    b: {
      type: "number",
      description: "The second number",
    },
  },
  required: ["a", "b"],
};

### Make the new function into a tool

In [11]:
const sumFunctionTool = new FunctionTool(sumNumbers, {
  name: "sumNumbers",
  description: "Use this function to sum two numbers",
  parameters: sumJSON,
});

### Make the router query engine into a tool

In [12]:
const queryEngineTool = new QueryEngineTool({
    queryEngine: queryEngine,
    metadata: {
        name: "react_and_dan_abramov_engine",
        description: "A tool that can answer questions about Dan Abramov and React",
    },
});

### Use the two tools

In [13]:
const agent = new OpenAIAgent({
    tools: [queryEngineTool, sumFunctionTool],
    verbose: true
})

### Let's ask it a question!

In [14]:
let response5 = await agent.chat({message:"What is React? Use a tool."})
console.log(response5.toString())

Added user message to memory!: What is React? Use a tool.
=== Calling Function ===
Calling function: react_and_dan_abramov_engine with args: {
  "query": "What is React?"
}
Got output React is a JavaScript library for building user interfaces. It was officially open-sourced in May 2013 at JSConf US, marking a significant turning point in its adoption and growth. React enables developers to create interactive and dynamic user interfaces for web applications. It has evolved over the years, with updates such as React Fiber, which introduced a new set of internal algorithms for rendering, and React 16.0, which was released to the public in September 2017. Additionally, React Native allows for native Android, iOS, and Universal Windows Platform development with React.
React is a JavaScript library for building user interfaces. It was officially open-sourced in May 2013 at JSConf US, marking a significant turning point in its adoption and growth. React enables developers to create interactiv

In [15]:
let response6 = await agent.chat({message:"What is 501 + 5?"})
console.log(response6.toString())

Added user message to memory!: What is 501 + 5?
=== Calling Function ===
Calling function: sumNumbers with args: {
  "a": 501,
  "b": 5
}
Got output 506
The sum of 501 and 5 is 506.
